In [1]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Users/sharattadimalla/github/deltalake/myenv/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/sharattadimalla/.ivy2/cache
The jars for the packages stored in: /Users/sharattadimalla/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ebd07664-842e-47fa-9f67-836c968296d4;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.1.1 in central
	found io.delta#delta-storage;2.1.1 in central
	found org.antlr#antlr4-runtime;4.8 in local-m2-cache
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in local-m2-cache
:: resolution report :: resolve 241ms :: artifacts dl 8ms
	:: modules in use:
	io.delta#delta-core_2.12;2.1.1 from central in [default]
	io.delta#delta-storage;2.1.1 from central in [default]
	org.antlr#antlr4-runtime;4.8 from local-m2-cache in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from local-m2-cache in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |


22/11/24 07:17:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
data = spark.range(0, 5)
data.show()
data.write.format("delta").mode("overwrite").save("/tmp/time-travel-delta-table")

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+



## delete from table

In [6]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, "/tmp/time-travel-delta-table")

deltaTable.delete("id == 2")

## history of the table

In [7]:
fullHistoryDF = deltaTable.history()
fullHistoryDF.show()

+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      2|2022-11-24 07:29:...|  null|    null|   DELETE|{predicate -> ["(...|null|    null|     null|          1|  Serializable|        false|{numRemovedFiles ...|        null|Apache-Spark/3.3....|
|      1|2022-11-24 07:27:...|  null|    null|    WRITE|{mode -> Overwrit...|null|    null|     null|          0|  Serializable|        false|{numFiles -> 6, n...|        null|Apache-Spark/3.3....|
|      0|2

## DELTA TIME TRAVEL USING `version`

In [8]:
v0df = spark.read.format("delta").option("versionAsOf", 0).load("/tmp/time-travel-delta-table")
v0df.show()

+---+
| id|
+---+
|  4|
|  0|
|  1|
|  2|
|  3|
+---+



In [10]:
v2df = spark.read.format("delta").option("versionAsOf", 2).load("/tmp/time-travel-delta-table")
v2df.show()

+---+
| id|
+---+
|  1|
|  3|
|  4|
|  0|
+---+



## DELTA TIME TRAVEL USING `TIMESTAMP`

In [12]:
t1df = spark.read.format("delta").option("timestamp", "2022-11-24 07:18").load("/tmp/time-travel-delta-table")
t1df.show()

+---+
| id|
+---+
|  1|
|  3|
|  4|
|  0|
+---+



In [13]:
t2df = spark.read.format("delta").option("timestamp", "2022-11-24 07:29").load("/tmp/time-travel-delta-table")
t2df.show()

+---+
| id|
+---+
|  1|
|  3|
|  4|
|  0|
+---+

